In [2]:
from google.colab import drive
from google.colab import files

import io
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# read a csv file from drive as a pd dataframe
df = pd.read_csv('/content/drive/MyDrive/NBA_PBP_2015-19.csv')


In [4]:
# Create new "Winner" column and set it to be "HOME" if WinningTeam = HomeTeam, otherwise "AWAY"
df['Winner'] = 'AWAY'
df.loc[df['WinningTeam'] == df['HomeTeam'], 'Winner'] = 'HOME'

In [5]:
df['Winner'].value_counts(normalize=True)

,proportion
Winner,
HOME,0.588332
AWAY,0.411668


In [6]:
df.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,...,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,Winner
0,/boxscores/201510270ATL.html,regular,Philips Arena Atlanta Georgia,October 27 2015,8:00 PM,DET,1,720,DET,Jump ball: A. Drummond vs. A. Horford (E. lyas...,...,NaN,NaN,NaN,NaN,NaN,NaN,A. Drummond - drumman01,A. Horford - horfoal01,E. İlyasova - ilyaser01,AWAY
1,/boxscores/201510270ATL.html,regular,Philips Arena Atlanta Georgia,October 27 2015,8:00 PM,DET,1,701,DET,A. Drummond misses 2-pt layup from 1 ft (block...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AWAY
2,/boxscores/201510270ATL.html,regular,Philips Arena Atlanta Georgia,October 27 2015,8:00 PM,DET,1,699,DET,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AWAY
3,/boxscores/201510270ATL.html,regular,Philips Arena Atlanta Georgia,October 27 2015,8:00 PM,DET,1,697,DET,NaN,...,NaN,NaN,K. Bazemore - bazemke01,bad pass,NaN,NaN,NaN,NaN,NaN,AWAY
4,/boxscores/201510270ATL.html,regular,Philips Arena Atlanta Georgia,October 27 2015,8:00 PM,DET,1,681,DET,M. Morris makes 2-pt jump shot from 13 ft (ass...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AWAY


In [7]:
df

KeyboardInterrupt: 

In [8]:
pd.set_option('future.no_silent_downcasting', True)

def add_halftime_winner(game_df):
  # Find row where AwayPlay is "End of 2nd Quarter"
  end_of_half = game_df[game_df['AwayPlay'].str.contains('End of 2nd quarter').fillna(False)]
  assert(len(end_of_half) > 0, f"No row found where AwayPlay is 'End of 2nd Quarter' for URL {df['URL'].values[0]}")
  home_points = end_of_half['HomeScore'].values[0]
  away_points = end_of_half['AwayScore'].values[0]
  pred_winner = 'HOME' if home_points >= away_points else 'AWAY'
  actual_winner = game_df['Winner'].values[0]
  return pd.Series({'HalftimeWinner': pred_winner, 'ActualWinner': actual_winner})

heuristic_pred = df.groupby('URL').apply(add_halftime_winner)

<>:6: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:6: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-8-2ef0df898cb6>:6: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(end_of_half) > 0, f"No row found where AwayPlay is 'End of 2nd Quarter' for URL {df['URL'].values[0]}")
<ipython-input-8-2ef0df898cb6>:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  heuristic_pred = df.groupby('URL').apply(add_halftime_winner)


In [10]:
heuristic_pred['correct_pred'] = (heuristic_pred['HalftimeWinner'] == heuristic_pred['ActualWinner'])
heuristic_pred['correct_pred'].value_counts(normalize=True)

,proportion
correct_pred,
True,0.728087
False,0.271913


In [61]:
# filter DF to only include rows where the URL is /boxscores/201510270ATL.html
df_ATL = df[df['URL'] == '/boxscores/201510270GSW.html']
halftime = df_ATL[df_ATL['AwayPlay'].str.contains('End of 2nd quarter').fillna(False)]
halftime['HomeScore'].values[0].dtype

<ipython-input-61-9896f870c954>:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  halftime = df_ATL[df_ATL['AwayPlay'].str.contains('End of 2nd quarter').fillna(False)]


dtype('int64')

In [35]:
df.columns

Index(['URL', 'GameType', 'Location', 'Date', 'Time', 'WinningTeam', 'Quarter',
       'SecLeft', 'AwayTeam', 'AwayPlay', 'AwayScore', 'HomeTeam', 'HomePlay',
       'HomeScore', 'Shooter', 'ShotType', 'ShotOutcome', 'ShotDist',
       'Assister', 'Blocker', 'FoulType', 'Fouler', 'Fouled', 'Rebounder',
       'ReboundType', 'ViolationPlayer', 'ViolationType', 'TimeoutTeam',
       'FreeThrowShooter', 'FreeThrowOutcome', 'FreeThrowNum', 'EnterGame',
       'LeaveGame', 'TurnoverPlayer', 'TurnoverType', 'TurnoverCause',
       'TurnoverCauser', 'JumpballAwayPlayer', 'JumpballHomePlayer',
       'JumpballPoss', 'Winner'],
      dtype='object')

In [36]:
# find rows where the string "End" is in every row of df
df_ATL[df_ATL['AwayPlay'].str.contains('End').fillna(False)]

<ipython-input-36-b184342ce581>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_ATL[df_ATL['AwayPlay'].str.contains('End').fillna(False)]


,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,...,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,Winner
1056,/boxscores/201510270GSW.html,regular,Oracle Arena Oakland California,October 27 2015,10:30 PM,GSW,1,0,NOP,End of 1st quarter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOME
1191,/boxscores/201510270GSW.html,regular,Oracle Arena Oakland California,October 27 2015,10:30 PM,GSW,2,0,NOP,End of 2nd quarter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOME
1328,/boxscores/201510270GSW.html,regular,Oracle Arena Oakland California,October 27 2015,10:30 PM,GSW,3,0,NOP,End of 3rd quarter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOME
1453,/boxscores/201510270GSW.html,regular,Oracle Arena Oakland California,October 27 2015,10:30 PM,GSW,4,0,NOP,End of 4th quarter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOME
1454,/boxscores/201510270GSW.html,regular,Oracle Arena Oakland California,October 27 2015,10:30 PM,GSW,4,0,NOP,End of Game,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOME


In [14]:
import re

def extract_and_average_percentages(text):
    # Regular expression to find all percentages in the format of a number followed by a %
    percentages = re.findall(r'(\d+\.\d+)?%', text)

    print("Extracted Percentages:", percentages)

    # Convert the percentages to float numbers by removing the '%' and dividing by 100
    percentage_values = [float(p.strip('%')) for p in percentages]

    # Calculate the average if there are any percentages found
    if percentage_values:
        return sum(percentage_values) / len(percentage_values)
    else:
        return 0  # Return 0 if no percentages are found

text = """Golden State	471-161-0	74.5%	8.2	-0.2
Philadelphia	320-120-0	72.7%	6.7	+0.5
San Antonio	281-108-0	72.2%	7.6	+0.6
LA Clippers	367-149-0	71.1%	6.7	+0.1
Milwaukee	416-169-0	71.1%	6.7	-0.5
Boston	470-192-0	71.0%	6.9	+0.2
Cleveland	320-132-0	70.8%	6.0	-0.7
Toronto	353-148-0	70.5%	6.3	0.0
Okla City	297-129-0	69.7%	7.0	+0.4
Denver	356-155-0	69.7%	5.4	-0.9
Houston	296-129-0	69.7%	6.6	-0.2
Utah	328-153-0	68.2%	7.0	+0.1
Portland	236-113-0	67.6%	5.2	+0.1
Miami	318-155-0	67.2%	4.7	-0.7
Orlando	147-72-0	67.1%	5.1	0.0
Phoenix	245-124-0	66.4%	5.2	-0.8
Brooklyn	187-97-0	65.9%	4.7	-0.7
Indiana	254-132-0	65.8%	5.0	-0.3
LA Lakers	238-125-0	65.6%	4.3	-1.5
Dallas	255-134-0	65.6%	5.2	-0.4
New York	182-97-0	65.2%	5.6	+0.7
Charlotte	171-92-0	65.0%	5.2	+0.2
Memphis	223-121-0	64.8%	5.0	-0.1
Chicago	164-99-0	62.4%	3.9	-0.9
Minnesota	218-133-0	62.1%	5.3	-0.6
Washington	177-113-0	61.0%	3.0	-1.9
New Orleans	199-128-0	60.9%	4.6	-0.5
Atlanta	214-138-0	60.8%	3.4	-1.9
Detroit	130-84-0	60.8%	4.1	-0.7
Sacramento	173-113-0	60.5%	3.7	-1.4"""

extract_and_average_percentages(text)

Extracted Percentages: ['74.5', '72.7', '72.2', '71.1', '71.1', '71.0', '70.8', '70.5', '69.7', '69.7', '69.7', '68.2', '67.6', '67.2', '67.1', '66.4', '65.9', '65.8', '65.6', '65.6', '65.2', '65.0', '64.8', '62.4', '62.1', '61.0', '60.9', '60.8', '60.8', '60.5']


66.86333333333334